# 2.2 The Art and Skill of Problem Formulation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1nE4Wrx1db9adNdhf8WfisH5yb-aZdwmT?usp=sharing)

A combination of practical insight and technical skill is required in order to recognize _which_ problems can be appropriately modeled in a linear programming format, and then to formulate those problems accurately. Because of the wide variety of problems that can be made to fall into the linear programming mold, it is difficult to give guidelines that are universally applicable to the process of problem formulation. Rather, problem formulation is an art that must be cultivated through practice and experience. Several examples are given to point the way, and to illustrate the creativity that is sometimes helpful in framing problems as linear programs. The exercises at the end of the chapter should then provide some of the practice necessary to develop the skill of formulating linear programming models.

<ul>

Example 2.2.1
<br />

A manufacturer of computer system components assembles two models of wireless routers, model A and model B. The amounts of materials and labor required for each assembly, and the total amounts available, are shown in the following table. The profits that can be realized from the sale of each router are $22 and $28 for models A and B, respectively, and we assume there is a market for as many routers as can be manufactured.
<br />
<br />

| Resources Required per Unit |     |     | Resources Available |
|-----------------------------|-----|-----|---------------------|
|                             |  A  |  B  |                     |
| Materials                   |  8  | 10  |        3400         |
| Labor                       |  2  | 3   |         960         |

<br />

The manufacturer would like to determine how many of each model to assemble in
order to maximize profits.
<br />
<br />

Formulation 2.2.1
<br />

Because the solution to this problem involves establishing the number of routers to be assembled, we define the decision variables as follows:

$$
\text{Let } x_A = \text{number of model A routers to be assembled}
$$

and

$$
x_B = \text{number of model B routers to be assembled}
$$

In order to maximize profits, we establish the objective criterion as

$$
\text{maximize } z = 22 x_A + 28 x_B
$$

Two types of resource limitations are in effect. The materials constraint is expressed by
the inequality

$$
8 x_A +  10 x_b \leq 3400
$$

and the labor constraint by

$$
2 x_A + 3 x_B \leq 960
$$

Finally, as it would be meaningless to have a negative number of terminals manufactured, we also include the constraints $x_A$ $\geq$ 0 and $x_B$ $\geq$ 0.

<br />
<br />

**Code Implementation**

In [ ]:
# Importing Pyomo environment
from pyomo.environ import * # Imports all components from the Pyomo environment, like Var, Param, Objective, Constraint, etc.

# Function to solve and display results of an optimization model
def solve_model(model, example_name):
    print("\n" + "="*40) # Prints a separator line for better output formatting.
    print(f" Solving {example_name} ") # Prints the name of the example being solved.
    print("="*40) # Prints another separator line.
    solver = SolverFactory('cbc') # Creates a solver object using the CBC solver.
    result = solver.solve(model) # Solves the given Pyomo model using the CBC solver.
    print("\nOptimal Solution:") # Prints a header for the optimal solution.
    for v in model.component_objects(Var, active=True): # Iterates through all active variables in the model.
        for index in v: # Iterates through any indices of the variable (if it's indexed).
            print(f"  {v[index]} = {v[index].value}") # Prints the variable name and its optimal value.
    print("\nObjective Value:", model.obj()) # Prints the optimal value of the objective function.

# Function to build the abstract Pyomo model for Example 2.2.1
def build_abstract_model_221():
    model = AbstractModel() # Creates an AbstractModel object, which is a Pyomo model that is defined without specific data values.
    model.xA = Var(within=NonNegativeReals) # Defines a decision variable `xA` representing the number of Model A routers produced, constrained to be non-negative.
    model.xB = Var(within=NonNegativeReals) # Defines a decision variable `xB` representing the number of Model B routers produced, constrained to be non-negative.
    model.obj_coeff_A = Param() # Defines a parameter `obj_coeff_A` for the objective function coefficient of `xA`.
    model.obj_coeff_B = Param() # Defines a parameter `obj_coeff_B` for the objective function coefficient of `xB`.
    model.materials_coeff_A = Param() # Defines a parameter `materials_coeff_A` for the materials constraint coefficient of `xA`.
    model.materials_coeff_B = Param() # Defines a parameter `materials_coeff_B` for the materials constraint coefficient of `xB`.
    model.materials_rhs = Param() # Defines a parameter `materials_rhs` for the right-hand side of the materials constraint.
    model.labor_coeff_A = Param() # Defines a parameter `labor_coeff_A` for the labor constraint coefficient of `xA`.
    model.labor_coeff_B = Param() # Defines a parameter `labor_coeff_B` for the labor constraint coefficient of `xB`.
    model.labor_rhs = Param() # Defines a parameter `labor_rhs` for the right-hand side of the labor constraint.

    # Defines the objective function rule
    def obj_rule(model):
        return model.obj_coeff_A * model.xA + model.obj_coeff_B * model.xB # Specifies the objective function to maximize profit.
    model.obj = Objective(rule=obj_rule, sense=maximize) # Creates the objective function object with the defined rule and maximization sense.

    # Defines the materials constraint rule
    def materials_rule(model):
        return model.materials_coeff_A * model.xA + model.materials_coeff_B * model.xB <= model.materials_rhs # Specifies the materials constraint.
    model.materials = Constraint(rule=materials_rule) # Creates the materials constraint object with the defined rule.

    # Defines the labor constraint rule
    def labor_rule(model):
        return model.labor_coeff_A * model.xA + model.labor_coeff_B * model.xB <= model.labor_rhs # Specifies the labor constraint.
    model.labor = Constraint(rule=labor_rule) # Creates the labor constraint object with the defined rule.
    return model # Returns the built abstract model.

# Data for Example 2.2.1
data_221 = {
    None: {
        'obj_coeff_A': {None: 22}, 'obj_coeff_B': {None: 28}, # Objective function coefficients for xA and xB
        'materials_coeff_A': {None: 8}, 'materials_coeff_B': {None: 10}, 'materials_rhs': {None: 3400}, # Materials constraint coefficients and RHS
        'labor_coeff_A': {None: 2}, 'labor_coeff_B': {None: 3}, 'labor_rhs': {None: 960} # Labor constraint coefficients and RHS
    }
}
model_221 = build_abstract_model_221().create_instance(data_221) # Creates a concrete model instance by loading data into the abstract model.
solve_model(model_221, "Example 2.2.1") # Solves the model instance and prints the results.


 Solving Example 2.2.1 

Optimal Solution:
  xA = 150.0
  xB = 220.0

Objective Value: 9460.0


Example 2.2.2

A space agency planning team wishes to set up a schedule for launching satellites over a period of three years. Experimental payloads are of two types (say, T1 and T2), and each launch carries only one experiment. Externally negotiated agency policies dictate that at most 88 of payload type T1 and 126 of type T2 can be supported. For each launch, type T1 payloads will operate successfully with probability 0.85 and type T2 payloads with probability 0.75. In order for the program to be viable, there must be a total of at least 60 successful deployments. The agency is paid \$ 1.5 million for each successful T1 payload, and \$ 1.2 million for each successful T2 payload. The costs to the agency to prepare and launch the two types of payloads are \$ 1.05 million for each T1 and \$ 0.7 million for each T2. One week of time must be devoted to the preparation of each T2 launch payload and two weeks are required for T1 launch payloads. The agency, while providing a public service, wishes to maximize its expected net income from the satellite program.

<br />
<br />

Formulation 2.2.2

Let $x_1 =$ number of satellites launched carrying a type T1 payload, and $x_2 =$ number of satellites launched carrying a type T2 payload. Income is realized only when launches are successful, but costs are incurred for all launches. Therefore, the expected net income is

$$
(1.5)(0.85)x_1 + (1.2)(0.75)x_2 - (1.05)x_1 - (0.7)x_2 \quad \text{million dollars}
$$

The objective is then to maximize $ z = 0.225x_1 + 0.2x_2$. Problem constraints in this case are of various types. Agency policies impose the two simple constraints

$$
x_1 \leq 88 \quad \text{and} \quad x_2 \leq 126
$$

The successful deployment quota yields the constraint

$$
0.85x_1 + 0.75x_2 \geq 60
$$

If we assume that 52 weeks per year (for three years) can be applied to the satellite program, then the launch preparation time constraint is

$$
2x_1 + x_2 \leq 156
$$

As in the previous example, we include the non-negativity constraints $x_1 \geq 0$ and $x_2 \geq 0$.

<br />
<br />

**Code Implementation**

In [ ]:
from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_222():
    model = AbstractModel()

    # Decision variables
    model.x1 = Var(within=NonNegativeReals)  # T1 payloads launched
    model.x2 = Var(within=NonNegativeReals)  # T2 payloads launched

    # Parameters for objective coefficients and constraints
    model.obj_coeff = Param(['x1', 'x2'], within=Any)
    model.policy_constraints = Param([1, 2], within=Any)  # Maximum values for x1 and x2
    model.successful_deployments_coeffs = Param([1, 2], within=Any)  # Coefficients for successful deployments
    model.preparation_time_coeffs = Param([1, 2], within=Any)  # Coefficients for preparation time constraint

    # Objective function
    def obj_rule(model):
        return model.obj_coeff['x1'] * model.x1 + model.obj_coeff['x2'] * model.x2
    model.obj = Objective(rule=obj_rule, sense=maximize)

    # Constraints
    # Policy constraints
    def policy_constraint1_rule(model):
        return model.x1 <= model.policy_constraints[1]
    model.policy_constraint1 = Constraint(rule=policy_constraint1_rule)

    def policy_constraint2_rule(model):
        return model.x2 <= model.policy_constraints[2]
    model.policy_constraint2 = Constraint(rule=policy_constraint2_rule)

    # Successful deployments constraint
    def successful_deployments_rule(model):
        return model.successful_deployments_coeffs[1] * model.x1 + model.successful_deployments_coeffs[2] * model.x2 >= 60
    model.successful_deployments = Constraint(rule=successful_deployments_rule)

    # Preparation time constraint
    def preparation_time_rule(model):
        return model.preparation_time_coeffs[1] * model.x1 + model.preparation_time_coeffs[2] * model.x2 <= 156
    model.preparation_time = Constraint(rule=preparation_time_rule)

    return model

data_222 = {
    None: {
        'obj_coeff': {'x1': 0.225, 'x2': 0.2},  # Objective function coefficients
        'policy_constraints': {1: 88, 2: 126},  # Maximum values for x1 and x2
        'successful_deployments_coeffs': {1: 0.85, 2: 0.75},  # Coefficients for successful deployments
        'preparation_time_coeffs': {1: 2, 2: 1},  # Coefficients for preparation time constraint
    }
}

# Instantiate the model
model_222 = build_abstract_model_222()
instance_222 = model_222.create_instance(data_222)
solve_model(instance_222, "Example 2.2.2")

ModuleNotFoundError: No module named 'pyomo'

Example 2.2.3

A company wishes to minimize its combined costs of production and inventory over a four-week time period. An item produced in a given week is available for consumption during that week, or it may be kept in inventory for use in later weeks. Initial inventory at the beginning of week 1 is 250 units. The minimum allowed inventory carried from one week to the next is 50 units. Unit production cost is \$15, and the cost of storing a unit from one week to the next is \$3. The following table shows production capacities and the demands that must be met during each week.

| Period | Production Capacity | Demand |
|--------|---------------------|--------|
| 1      | 800                 | 900    |
| 2      | 700                 | 600    |
| 3      | 600                 | 800    |
| 4      | 800                 | 600    |

A minimum production of 500 items per week must be maintained. Inventory costs are not applied to items remaining at the end of the fourth production period, nor is the minimum inventory restriction applied after this final period.

<br />
<br />

Formulation 2.2.3

Let $x_i$ be the number of units produced during the $i$-th week, for $i = 1, \ldots, 4$. The formulation is somewhat more manageable if we let $A_i$ denote the number of items remaining at the end of each week (accounting for those held over from previous weeks, those produced during the current week, and those consumed during the current week). Note that the $A_i$ values are not decision variables, but merely serve to simplify our written formulation. Thus,

$$
A_1 = 250 + x_1 - 900
$$

$$
A_2 = A_1 + x_2 - 600
$$

$$
A_3 = A_2 + x_3 - 800
$$

$$
A_4 = A_3 + x_4 - 600
$$

The objective is to minimize

$$
z = 15(x_1 + x_2 + x_3 + x_4) + 3(A_1 + A_2 + A_3)
$$

Minimum inventory constraints are expressed as $A_i \geq 50$ for $i=1,2,3$, and $A_4 \geq 0$. Production capacities and minima during each period are enforced with the constraints

$$
500 \leq x_1 \leq 700
$$

$$
500 \leq x_2 \leq 700
$$

$$
500 \leq x_3 \leq 600
$$

$$
500 \leq x_4 \leq 800
$$

Finally, $x_i \geq 0$ for $i=1, \ldots, 4$.
<br />
<br />

**Code Implementation**

In [ ]:
from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_223():
    model = AbstractModel()
    model.T = Set()
    model.T_inv = Set()
    model.initial_inventory = Param()
    model.demand = Param(model.T)
    model.capacity = Param(model.T)
    model.production_cost = Param()
    model.inventory_cost = Param()
    model.min_production = Param()
    model.min_inventory = Param()

    model.x = Var(model.T, within=NonNegativeReals)
    model.A = Var(model.T_inv, within=NonNegativeReals)

    def obj_rule(model):
        return sum(model.production_cost * model.x[t] for t in model.T) + \
               sum(model.inventory_cost * model.A[t] for t in model.T_inv)
    model.obj = Objective(rule=obj_rule, sense=minimize)

    def inventory_balance_rule(model, t):
        if t == 1:
            return model.A[t] == model.initial_inventory + model.x[t] - model.demand[t]
        else:
            if t <= max(model.T_inv):
                return model.A[t-1] + model.x[t] - model.demand[t] == model.A[t]
            else:
                return model.A[t-1] + model.x[t] - model.demand[t] == 0
    model.inventory_balance = Constraint(model.T, rule=inventory_balance_rule)

    model.production_min = Constraint(model.T, rule=lambda m, t: m.x[t] >= m.min_production)
    model.production_max = Constraint(model.T, rule=lambda m, t: m.x[t] <= m.capacity[t])
    model.inventory_min = Constraint(model.T_inv, rule=lambda m, t: m.A[t] >= m.min_inventory)
    return model

data_223 = {None: {
    'T': {1, 2, 3, 4}, 'T_inv': {1, 2, 3},
    'initial_inventory': {None: 250},
    'demand': {1: 900, 2: 600, 3: 800, 4: 600},
    'capacity': {1: 800, 2: 700, 3: 600, 4: 800},
    'production_cost': {None: 15}, 'inventory_cost': {None: 3},
    'min_production': {None: 500}, 'min_inventory': {None: 50}
}}
model_223 = build_abstract_model_223().create_instance(data_223)
solve_model(model_223, "Example 2.2.3")


 Solving Example 2.2.3 

Optimal Solution:
  x[1] = 800.0
  x[2] = 700.0
  x[3] = 600.0
  x[4] = 550.0
  A[1] = 150.0
  A[2] = 250.0
  A[3] = 50.0

Objective Value: 41100.0


Example 2.2.4

A mixture of freeze-dried vegetables is to be composed of beans, corn, broccoli, cabbage, and potatoes. The mixture is to contain (by weight) at most 40% beans and at most 32% potatoes. The mixture should contain at least 5 grams iron, 36 grams phosphorus, and 28 grams calcium. The nutrients in each vegetable and the costs are shown in the following table.

| Vegetable | Iron (mg/lb) | Phosphorus (mg/lb) | Calcium (mg/lb) | Cost (cents/lb) |
|-----------|--------------|--------------------|-----------------|-----------------|
| Beans     | 0.5          | 10                 | 200             | 20              |
| Corn      | 0.5          | 20                 | 280             | 18              |
| Broccoli  | 1.2          | 40                 | 800             | 32              |
| Cabbage   | 0.3          | 30                 | 420             | 28              |
| Potatoes  | 0.4          | 50                 | 360             | 16              |

The amount of each vegetable to include should be determined so that the cost of the mixture is minimized.
<br />
<br />

Formulation 2.2.4

Let $x_1, x_2, x_3, x_4, x_5$ be the number of pounds of beans, corn, broccoli, cabbage, and potatoes, respectively. To minimize the cost of the mixture, we wish to minimize $z = 20x_1 + 18x_2 + 32x_3 + 28x_4 + 16x_5$. The percentage of beans in the mixture is $\frac{x_1}{x_1 + x_2 + x_3 + x_4 + x_5}$, and must be less than 40%. Therefore,

$$
x_1 \leq 0.4 (x_1 + x_2 + x_3 + x_4 + x_5)
$$

and similarly the potato restriction can be written as

$$
x_5 \leq 0.32 (x_1 + x_2 + x_3 + x_4 + x_5)
$$

To achieve the required level of nutrients, we have three constraints (for iron, phosphorus, and calcium, respectively):

$$
0.5x_1 + 0.5x_2 + 1.2x_3 + 0.3x_4 + 0.4x_5 \geq 5000
$$

$$
10x_1 + 20x_2 + 40x_3 + 30x_4 + 50x_5 \geq 36000
$$

$$
200x_1 + 280x_2 + 800x_3 + 420x_4 + 360x_5 \geq 28000
$$

Negative amounts are not possible, so $x_i \geq 0$ for $i=1, \ldots, 5$.
<br />
<br />

**Code Implementation**

In [ ]:
from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_224():
    model = AbstractModel()

    # Decision variables for pounds of beans, corn, broccoli, cabbage, and potatoes
    model.x1 = Var(within=NonNegativeReals)
    model.x2 = Var(within=NonNegativeReals)
    model.x3 = Var(within=NonNegativeReals)
    model.x4 = Var(within=NonNegativeReals)
    model.x5 = Var(within=NonNegativeReals)

    # Parameters
    model.total_ingredients = Param(within=NonNegativeReals, default=1)  # Total of all ingredients
    model.iron_requirement = Param(within=NonNegativeReals, default=5000)
    model.phosphorus_requirement = Param(within=NonNegativeReals, default=36000)
    model.calcium_requirement = Param(within=NonNegativeReals, default=28000)

    # Objective function: Minimize cost
    def obj_rule(model):
        return 20*model.x1 + 18*model.x2 + 32*model.x3 + 28*model.x4 + 16*model.x5
    model.obj = Objective(rule=obj_rule, sense=minimize)

    # Constraints

    # Ingredient limits
    def ingredient_limit_1_rule(model):
        return model.x1 <= 0.4 * (model.x1 + model.x2 + model.x3 + model.x4 + model.x5)
    model.ingredient_limit_1 = Constraint(rule=ingredient_limit_1_rule)

    def ingredient_limit_2_rule(model):
        return model.x5 <= 0.32 * (model.x1 + model.x2 + model.x3 + model.x4 + model.x5)
    model.ingredient_limit_2 = Constraint(rule=ingredient_limit_2_rule)

    # Nutritional requirements
    def iron_rule(model):
        return 0.5 * model.x1 + 0.5 * model.x2 + 1.2 * model.x3 + 0.3 * model.x4 + 0.4 * model.x5 >= model.iron_requirement
    model.iron_constraint = Constraint(rule=iron_rule)

    def phosphorus_rule(model):
        return 10 * model.x1 + 20 * model.x2 + 40 * model.x3 + 30 * model.x4 + 50 * model.x5 >= model.phosphorus_requirement
    model.phosphorus_constraint = Constraint(rule=phosphorus_rule)

    def calcium_rule(model):
        return 200 * model.x1 + 280 * model.x2 + 800 * model.x3 + 420 * model.x4 + 360 * model.x5 >= model.calcium_requirement
    model.calcium_constraint = Constraint(rule=calcium_rule)

    return model

data_224 = {
    None: {
        'total_ingredients': {None: 1},  # Mapping required for each parameter
        'iron_requirement': {None: 5000},
        'phosphorus_requirement': {None: 36000},
        'calcium_requirement': {None: 28000}
    }
}

# Instantiate the model
model_224 = build_abstract_model_224()
instance_224 = model_224.create_instance(data_224)
solve_model(instance_224, "Example 2.2.4")


 Solving Example 2.2.4 

Optimal Solution:
  x1 = 0.0
  x2 = 0.0
  x3 = 4166.6667
  x4 = 0.0
  x5 = 0.0

Objective Value: 133333.3344


Example 2.2.5

A saw mill makes two products for log home kits: fir logs and spruce logs which can be sold at profits of \$4 and \$5, respectively. Spruce logs require two units of processing time on the bark peeler and six units of time on a slab saw. Fir logs require three units of time on the peeler and five units on the slab saw. Each then requires two units of time on the planer. Because of maintenance requirements and labor restrictions, the bark peeler is available 10 hours per day, the slab saw 12 hours per day, and the planer 14 hours per day. Bark and sawdust are by-products of these operations. All the bark can be put through a chipper and sold in unlimited quantities to a nearby horticulture supplier. Dried fir sawdust can be directed to a similar market, at a net profit of \$0.38 per processed log. Limited amounts of the spruce sawdust can be made into marketable pressed wood products, but the rest must be destroyed. For each spruce log produced, enough sawdust (five pounds) is generated to make three pressed wood products, which after manufacturing can be sold at a unit profit of \$0.22. However, the market can absorb only 60 of the pressed wood products per day and the remaining spruce sawdust must be destroyed at a cost of \$0.15 per pound. The saw mill wishes to make the largest possible profit, considering the cost of destroying the unusable sawdust.
<br />
<br />

Formulation 2.2.5

The formulation of this problem cannot follow exactly the pattern established in previous examples because the profits to be maximized are not a linear function of the number of logs of each type produced. Spruce log production creates a by-product that is useful and profitable only up to a point, and thereafter any excess must be destroyed at a cost that diminishes total profits. Thus, profits are not a strictly increasing function of production levels. We can still let

$$
x_1 = \text{number of fir logs produced}
$$

$$
x_2 = \text{number of spruce logs produced}
$$

Because sawdust contributes nonlinearly to profits, we treat it in two parts and let

$$
x_3 = \text{number of pounds of spruce sawdust used}
$$

$$
x_4 = \text{number of pounds of spruce sawdust destroyed}
$$

Direct profit from the sale of logs is $4x_1 + 5x_2$. All the bark can be sold at a profit in unlimited quantities, therefore, although this affects the amount of profit, it does not affect our decision on how many logs of each type to produce. Fir sawdust brings in \$0.38 for each processed log, or \$0.38 $x_1$. For each $\frac{x_3}{5}$ spruce logs produced, there is enough sawdust to make three products at a profit of \$0.22 each, if there is a market. Unmarketable spruce sawdust costs \$0.15 $x_4$ to destroy. The objective is, therefore, to maximize

$$
z = 4x_1 + 5x_2 + 0.38x_1 +  \frac{3}{5} (0.22) x_3 - 0.15 x_4
$$

Relating the number of logs produced to pounds of sawdust by-product, we obtain the constraint

$$
5x_2 = x_3 + x_4
$$

Limitations on demand for the pressed wood product are expressed by

$$
\frac{3}{5} x_3 \leq 60
$$

Constraints on availability of machinery are straightforward. For the bark peeler,

$$
3x_1 + 2x_2 \leq 10
$$

On the slab saw,

$$
5x_1 + 6x_2 \leq 12
$$

And on the planer,

$$
2x_1 + 2x_2 \leq 14
$$

Because all production levels are non-negative, we also require $x_1 \geq 0$, $x_2 \geq 0$, $x_3 \geq 0$, and $x_4 \geq 0$.
<br />
<br />


**Code Implementation**

In [ ]:
from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_225():
    model = AbstractModel()

    # Decision variables
    model.x1 = Var(within=NonNegativeReals)  # Fir logs produced
    model.x2 = Var(within=NonNegativeReals)  # Spruce logs produced
    model.x3 = Var(within=NonNegativeReals)  # Pounds of spruce sawdust used
    model.x4 = Var(within=NonNegativeReals)  # Pounds of spruce sawdust destroyed

    # Parameters for objective coefficients and constraints
    model.obj_coeff = Param(['x1', 'x2', 'x3', 'x4'], within=Any)
    model.machine_time_constraints = Param([1, 2, 3], within=Any)  # For machine time constraints
    model.sawdust_balance_coeffs = Param([1, 2], within=Any)  # Sawdust balance coefficients

    # Objective function
    def obj_rule(model):
        return model.obj_coeff['x1'] * model.x1 + model.obj_coeff['x2'] * model.x2 + \
               model.obj_coeff['x3'] * model.x3 + model.obj_coeff['x4'] * model.x4
    model.obj = Objective(rule=obj_rule, sense=maximize)

    # Constraints for sawdust balance
    def sawdust_balance_rule(model):
        return model.sawdust_balance_coeffs[1] * model.x2 == model.x3 + model.x4
    model.sawdust_balance = Constraint(rule=sawdust_balance_rule)

    # Market limit for pressed wood
    def market_limit_rule(model):
        return (3 / 5) * model.x3 <= 60
    model.market_limit = Constraint(rule=market_limit_rule)

    # Machine time constraints
    def machine_time1_rule(model):
        return 3 * model.x1 + 2 * model.x2 <= model.machine_time_constraints[1]
    model.machine_time1 = Constraint(rule=machine_time1_rule)

    def machine_time2_rule(model):
        return 5 * model.x1 + 6 * model.x2 <= model.machine_time_constraints[2]
    model.machine_time2 = Constraint(rule=machine_time2_rule)

    def machine_time3_rule(model):
        return 2 * model.x1 + 2 * model.x2 <= model.machine_time_constraints[3]
    model.machine_time3 = Constraint(rule=machine_time3_rule)

    return model

data_225 = {
    None: {
        'obj_coeff': {'x1': 4.38, 'x2': 5, 'x3': 0.66, 'x4': -0.15},  # Objective function coefficients
        'machine_time_constraints': {1: 10, 2: 12, 3: 14},  # Machine time limits for the constraints
        'sawdust_balance_coeffs': {1: 5, 2: 1},  # Sawdust balance coefficients
    }
}

# Instantiate the model
model_225 = build_abstract_model_225()
instance_225 = model_225.create_instance(data_225)
solve_model(instance_225, "Example 2.2.5")


 Solving Example 2.2.5 

Optimal Solution:
  x1 = 0.0
  x2 = 2.0
  x3 = 10.0
  x4 = 0.0

Objective Value: 16.6


Example 2.2.6

A dual processor computing facility is to be dedicated to administrative and scientific application jobs for at least 10 hours each day. Administrative jobs require two seconds of execution time on processor 1 and six seconds on processor 2, while scientific jobs require five seconds on processor one and three seconds on processor 2. A scheduler must choose how many of each type of job (administrative and scientific) to execute, in such a way as to minimize the amount of time that the system is occupied with these jobs. The system is considered to be occupied even if one processor is idle. (Assume that the sequencing of the jobs on each processor is not an issue here, just the selection of how many of each type of job.)
<br />
<br />

Formulation 2.2.6

Let $x_1$ and $x_2$ denote, respectively, the number of administrative and scientific jobs selected for execution on the dual processor system. Because policies require that each processor be available for at least 10 hours, we must write the two constraints as:

$$
2x_1 + 5x_2 \geq 10 \times 3600 \quad \text{(Processor 1)}
$$

$$
6x_1 + 3x_2 \geq 10 \times 3600 \quad \text{(Processor 2)}
$$

and

$$
x_1 \geq 0, \quad x_2 \geq 0
$$

The system is considered occupied as long as either processor is busy. Therefore, to minimize the completion time for the set of jobs, we must

$$
\min (\max \{ 2x_1 + 5x_2, \; 6x_1 + 3x_2 \})
$$

This nonlinear objective can be made linear if we introduce a new variable $x_3$, where

$$
x_3 = \max \{ 2x_1 + 5x_2, \; 6x_1 + 3x_2 \} \geq 0
$$

Now if we require

$$
x_3 \geq 2x_1 + 5x_2, \quad x_3 \geq 6x_1 + 3x_2
$$

and make our objective to minimize $x_3$, we have the desired linear formulation.

</ul>
<br />

**Code Implementation**

In [ ]:
from pyomo.environ import * # importing Pyomo environment

def solve_model(model, example_name):
    print("\n" + "="*40)
    print(f" Solving {example_name} ")
    print("="*40)
    solver = SolverFactory('cbc')
    result = solver.solve(model)
    print("\nOptimal Solution:")
    for v in model.component_objects(Var, active=True):
        for index in v:
            print(f"  {v[index]} = {v[index].value}")
    print("\nObjective Value:", model.obj())

def build_abstract_model_226():
    model = AbstractModel()

    # Decision variables
    model.x1 = Var(within=NonNegativeReals)
    model.x2 = Var(within=NonNegativeReals)
    model.x3 = Var(within=NonNegativeReals)  # The maximum time variable

    # Parameters for processor coefficients (indexed by 1 and 2 for x1 and x2)
    model.processor1_coeffs = Param([1, 2], within=Any)
    model.processor2_coeffs = Param([1, 2], within=Any)
    model.processor1_min = Param(within=Any)
    model.processor2_min = Param(within=Any)

    # Constraints for processor time (ensuring both expressions are >= 36,000)
    def processor1_constraint_rule(model):
        return model.processor1_coeffs[1] * model.x1 + model.processor1_coeffs[2] * model.x2 >= model.processor1_min
    model.processor1_constraint = Constraint(rule=processor1_constraint_rule)

    def processor2_constraint_rule(model):
        return model.processor2_coeffs[1] * model.x1 + model.processor2_coeffs[2] * model.x2 >= model.processor2_min
    model.processor2_constraint = Constraint(rule=processor2_constraint_rule)

    # Linearization for the max function: x3 should be greater than both expressions
    def linearization_constraint1_rule(model):
        return model.x3 >= model.processor1_coeffs[1] * model.x1 + model.processor1_coeffs[2] * model.x2
    model.linearization_constraint1 = Constraint(rule=linearization_constraint1_rule)

    def linearization_constraint2_rule(model):
        return model.x3 >= model.processor2_coeffs[1] * model.x1 + model.processor2_coeffs[2] * model.x2
    model.linearization_constraint2 = Constraint(rule=linearization_constraint2_rule)

    # Objective function: Minimize x3
    def obj_rule(model):
        return model.x3  # Minimize the maximum time
    model.obj = Objective(rule=obj_rule)

    return model

data_226 = {None: {
    'processor1_coeffs': {1: 1.0, 2: 2.0}, 'processor1_min': {None: 36000},
    'processor2_coeffs': {1: 2.0, 2: 1.0}, 'processor2_min': {None: 18000}
}}
model_226 = build_abstract_model_226()
instance_226 = model_226.create_instance(data_226)
solve_model(instance_226, "Example 2.2.6")


 Solving Example 2.2.6 

Optimal Solution:
  x1 = 0.0
  x2 = 18000.0
  x3 = 36000.0

Objective Value: 36000.0


2.2.1 Integer and Nonlinear Models

There are many problems that appear to fall into the framework of linear programming problem formulations. In some problems, the decision variable values are meaningful only if they are integer values. (For example, it is not possible to launch a fractional number of satellites or to transport a fractional number of products.) In such cases, the linear programming model is modified by adding the requirement that some or all of the variables must take integer values. This gives rise to an **integer programming** problem.

In other cases, the objective function or some of the constraints may be nonlinear functions of the decision variables. When this occurs, the resulting model is called a **nonlinear programming** problem. Nonlinear programming models are generally more difficult to solve than linear programming models, but many practical problems do require such formulations.